In [108]:
# Import des librairies

import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score

In [104]:
# ÉTUDE DU DATASET

# Variable cible : final_result
learning_analytics_B = pd.read_csv("learning_analytics_B.csv" , sep = ',', header = 0, index_col = 0)
learning_analytics_J = pd.read_csv("learning_analytics_J.csv" , sep = ',', header = 0, index_col = 0)

def resumetable(df):
    print("Format de la base : {}".format(df.shape))
    summary = pd.DataFrame(df.dtypes, columns=["dtypes"]).reset_index() #initialisation
    summary["Name"] = summary["index"] #Nom des colonnes
    summary = summary[["Name","dtypes"]]
    summary["Missing"] = df.isnull().sum().values #Nombre de valeurs manquantes
    summary["Uniques"] = df.nunique().values #Nb de modalités uniques
    summary["First Value"] = df.iloc[0].values #Exemple des 3 premieres lignes
    summary["Second Value"] = df.iloc[1].values 
    summary["Third Value"] = df.iloc[2].values 

    return summary

print('•\tSTATS MACROS SUR LA TABLE : learning_analytics_B')
display( resumetable(learning_analytics_B) )

print("•\tSTATS MACROS SUR LA TABLE : learning_analytics_J")
display( resumetable(learning_analytics_J) )

•	STATS MACROS SUR LA TABLE : learning_analytics_B
Format de la base : (9798, 17)


,Name,dtypes,Missing,Uniques,First Value,Second Value,Third Value
0,final_result,object,0,4,Fail,Pass,Fail
1,disability,object,0,2,N,N,N
2,module_presentation_length,int64,0,3,241,240,240
3,gender,object,0,2,M,F,M
4,highest_education,object,0,5,HE Qualification,Lower Than A Level,A Level or Equivalent
5,imd_band,object,399,10,80-90%,90-100%,20-30%
6,num_of_prev_attempts,int64,0,7,1,0,0
7,region,object,0,13,Scotland,East Anglian Region,London Region
8,studied_credits,int64,0,41,120,60,120
9,age_band,object,0,3,55<=,35-55,35-55


•	STATS MACROS SUR LA TABLE : learning_analytics_J
Format de la base : (16041, 17)


,Name,dtypes,Missing,Uniques,First Value,Second Value,Third Value
0,final_result,object,0,4,Pass,Withdrawn,Fail
1,disability,object,0,2,N,N,N
2,module_presentation_length,int64,0,4,268,268,262
3,gender,object,0,2,F,F,M
4,highest_education,object,0,5,A Level or Equivalent,Lower Than A Level,Lower Than A Level
5,imd_band,object,599,10,0-10%,20-30%,70-80%
6,num_of_prev_attempts,int64,0,7,0,1,1
7,region,object,0,13,West Midlands Region,Wales,East Anglian Region
8,studied_credits,int64,0,43,60,120,60
9,age_band,object,0,3,35-55,0-35,0-35


In [105]:
# DATA PREPARATION

print('•\tDATA PREPARATION')

# remplissage des null de imb_band par 40-50% 
learning_analytics_B['imd_band'] = learning_analytics_B['imd_band'].fillna('40-50%')
learning_analytics_J['imd_band'] = learning_analytics_J['imd_band'].fillna('40-50%')

# remplissage des null de la moyenne pondérée par la moyenne
learning_analytics_B['Moyenne pondérée de score'] = learning_analytics_B['Moyenne pondérée de score'].fillna(learning_analytics_B['Moyenne de score'])
learning_analytics_J['Moyenne pondérée de score'] = learning_analytics_J['Moyenne pondérée de score'].fillna(learning_analytics_J['Moyenne de score'])

# remplissage des null de Nombre de date par 0 
learning_analytics_B['Nombre de date'] = learning_analytics_B['Nombre de date'].fillna(0)
learning_analytics_J['Nombre de date'] = learning_analytics_J['Nombre de date'].fillna(0)

# remplissage des null de sum_click par 0
learning_analytics_B['sum_click'] = learning_analytics_B['sum_click'].fillna(0)
learning_analytics_J['sum_click'] = learning_analytics_J['sum_click'].fillna(0)

# remplissage des null de Nombre de activity_type par 0
learning_analytics_B['Nombre de activity_type'] = learning_analytics_B['Nombre de activity_type'].fillna(0)
learning_analytics_J['Nombre de activity_type'] = learning_analytics_J['Nombre de activity_type'].fillna(0)

# On ne prend pas les moyennes de score ni les dates d'enregistrement null
learning_analytics_B = learning_analytics_B.dropna(subset=['Moyenne de score', 'date_registration'])
learning_analytics_J = learning_analytics_J.dropna(subset=['Moyenne de score', 'date_registration'])

print('•\tDATA PREPARATION : OK')

•	DATA PREPARATION
•	DATA PREPARATION : OK


In [106]:
print('\t○\tSTATS MACROS APRES LA DATA PREP : learning_analytics_B')
display( resumetable(learning_analytics_B) )

print("\t○\tSTATS MACROS  APRES LA DATA PREP : learning_analytics_J")
display( resumetable(learning_analytics_J) )

	○	STATS MACROS APRES LA DATA PREP : learning_analytics_B
Format de la base : (9786, 17)


,Name,dtypes,Missing,Uniques,First Value,Second Value,Third Value
0,final_result,object,0,4,Fail,Pass,Fail
1,disability,object,0,2,N,N,N
2,module_presentation_length,int64,0,3,241,240,240
3,gender,object,0,2,M,F,M
4,highest_education,object,0,5,HE Qualification,Lower Than A Level,A Level or Equivalent
5,imd_band,object,0,10,80-90%,90-100%,20-30%
6,num_of_prev_attempts,int64,0,7,1,0,0
7,region,object,0,13,Scotland,East Anglian Region,London Region
8,studied_credits,int64,0,41,120,60,120
9,age_band,object,0,3,55<=,35-55,35-55


	○	STATS MACROS  APRES LA DATA PREP : learning_analytics_J
Format de la base : (16026, 17)


,Name,dtypes,Missing,Uniques,First Value,Second Value,Third Value
0,final_result,object,0,4,Pass,Withdrawn,Fail
1,disability,object,0,2,N,N,N
2,module_presentation_length,int64,0,4,268,268,262
3,gender,object,0,2,F,F,M
4,highest_education,object,0,5,A Level or Equivalent,Lower Than A Level,Lower Than A Level
5,imd_band,object,0,10,0-10%,20-30%,70-80%
6,num_of_prev_attempts,int64,0,7,0,1,1
7,region,object,0,13,West Midlands Region,Wales,East Anglian Region
8,studied_credits,int64,0,43,60,120,60
9,age_band,object,0,3,35-55,0-35,0-35


In [107]:
    # Création du vecteur label
label_learning_analytics_B = learning_analytics_B['final_result']
label_learning_analytics_J = learning_analytics_J['final_result']
    # Suprression de la colonne target : final_result
del learning_analytics_B['final_result']
del learning_analytics_J['final_result']

In [133]:
from sklearn import linear_model

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import OneHotEncoder

# get dummies à faire
def model_final_result(label_learning_analytics, df_learning_analytics):
    
    #vecorizer = DictVectorizer()
    #df_learning_analytics = vectorizer.fit_transform(df_learning_analytics)

    # Split the data into training/testing sets
    #X_train = df_learning_analytics[:-20]
    #X_test = df_learning_analytics[-20:]
    # Split the targets into training/testing sets
    #y_train = label_learning_analytics[:-20]
    #y_test = label_learning_analytics[-20:]

    df_learning_analytics['disability'] = df_learning_analytics['disability'].astype(str)
    df_learning_analytics['module_presentation_length'] = df_learning_analytics['module_presentation_length'].astype(str)
    df_learning_analytics['gender'] = df_learning_analytics['gender'].astype(str)
    df_learning_analytics['highest_education'] = df_learning_analytics['highest_education'].astype(str)
    df_learning_analytics['imd_band'] = df_learning_analytics['imd_band'].astype(str)
    df_learning_analytics['num_of_prev_attempts'] = df_learning_analytics['num_of_prev_attempts'].astype(str)
    df_learning_analytics['region'] = df_learning_analytics['region'].astype(str)
    df_learning_analytics['studied_credits'] = df_learning_analytics['studied_credits'].astype(str)
    df_learning_analytics['age_band'] = df_learning_analytics['age_band'].astype(str)
    df_learning_analytics['Nombre de date_submitted'] = df_learning_analytics['Nombre de date_submitted'].astype(str)
    df_learning_analytics['Moyenne de score'] = df_learning_analytics['Moyenne de score'].astype(str)
    df_learning_analytics['Moyenne pondérée de score'] = df_learning_analytics['Moyenne pondérée de score'].astype(str)
    df_learning_analytics['date_registration'] = df_learning_analytics['date_registration'].astype(str)
    df_learning_analytics['Nombre de date'] = df_learning_analytics['Nombre de date'].astype(str)
    df_learning_analytics['sum_click'] = df_learning_analytics['sum_click'].astype(str)
    df_learning_analytics['Nombre de activity_type'] = df_learning_analytics['Nombre de activity_type'].astype(str)

    X = df_learning_analytics
    y = label_learning_analytics
    
    # Convert the label vector to one-hot encoding
    #y = to_categorical(y)
    
    X = OneHotEncoder().fit_transform(X).toarray()
    
    # Split into training and test
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # Create linear regression object
    regr = linear_model.LinearRegression()

    # Train the model using the training sets
    regr.fit(X_train, y_train)

    # Make predictions using the testing set
    y_pred = regr.predict(X_test)

    # The coefficients
    print("Coefficients: \n", regr.coef_)
    # The mean squared error
    print("Mean squared error: %.2f" % mean_squared_error(y_test, y_pred))
    # The coefficient of determination: 1 is perfect prediction
    print("Coefficient of determination: %.2f" % r2_score(y_test, y_pred))

    # Plot outputs
    plt.scatter(X_test, y_test, color="black")
    plt.plot(X_test, y_pred, color="blue", linewidth=3)

    plt.xticks(())
    plt.yticks(())

    plt.show()
    
model_final_result(label_learning_analytics_B, learning_analytics_B)

ValueError: could not convert string to float: 'Fail'